## 'Dogs vs Cats' in Keras/Tensorflow

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from keras import backend as K

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input
from keras.optimizers import Adam

Using TensorFlow backend.


In [ ]:
PATH = "../data/dogscats/"
sz,bs=224,64
train_data_dir,validation_data_dir = f'{PATH}train',f'{PATH}valid'

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    shear_range=0.2, zoom_range=0.2, rotation_range=10, brightness_range=(0.8,1.2), horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(sz, sz), batch_size=bs, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    target_size=(sz, sz), shuffle=False,
    batch_size=bs, class_mode='binary')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

/home/jhoward/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [ ]:
lr,num_workers = 1e-3,16

model = Model(inputs=base_model.input, outputs=predictions)
opt = Adam(lr=lr)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
%%time
model.fit_generator(train_generator, train_generator.n // bs, epochs=6, workers=num_workers,
        validation_data=validation_generator, validation_steps=validation_generator.n // bs)

Epoch 1/6
359/359 [==============================] - 88s 246ms/step - loss: 0.1135 - acc: 0.9572 - val_loss: 0.0497 - val_acc: 0.9825
Epoch 2/6
359/359 [==============================] - 85s 236ms/step - loss: 0.0621 - acc: 0.9740 - val_loss: 0.0665 - val_acc: 0.9795
Epoch 3/6
359/359 [==============================] - 85s 236ms/step - loss: 0.0545 - acc: 0.9799 - val_loss: 0.1027 - val_acc: 0.9720
Epoch 4/6
359/359 [==============================] - 84s 234ms/step - loss: 0.0502 - acc: 0.9798 - val_loss: 0.0617 - val_acc: 0.9820
Epoch 5/6
359/359 [==============================] - 84s 233ms/step - loss: 0.0470 - acc: 0.9813 - val_loss: 0.0761 - val_acc: 0.9795
Epoch 6/6
359/359 [==============================] - 84s 234ms/step - loss: 0.0409 - acc: 0.9837 - val_loss: 0.0799 - val_acc: 0.9795
CPU times: user 37min 20s, sys: 1min 44s, total: 39min 5s
Wall time: 8min 29s


In [ ]:
model.save_weights(f'{PATH}/models/keras_0.h5')

In [ ]:
model.load_weights(f'{PATH}/models/keras_0.h5')

In [ ]:
for layer in model.layers: layer.trainable = True
K.set_value(opt.lr , lr/100)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
%%time
model.fit_generator(train_generator, train_generator.n // bs, epochs=6, workers=num_workers,
        validation_data=validation_generator, validation_steps=validation_generator.n // bs)

Epoch 1/6
359/359 [==============================] - 184s 514ms/step - loss: 0.0309 - acc: 0.9885 - val_loss: 0.0342 - val_acc: 0.9865
Epoch 2/6
359/359 [==============================] - 174s 484ms/step - loss: 0.0121 - acc: 0.9965 - val_loss: 0.0251 - val_acc: 0.9910
Epoch 3/6
359/359 [==============================] - 173s 481ms/step - loss: 0.0092 - acc: 0.9970 - val_loss: 0.0363 - val_acc: 0.9870
Epoch 4/6
359/359 [==============================] - 172s 480ms/step - loss: 0.0069 - acc: 0.9977 - val_loss: 0.0238 - val_acc: 0.9900
Epoch 5/6
359/359 [==============================] - 173s 483ms/step - loss: 0.0051 - acc: 0.9985 - val_loss: 0.0236 - val_acc: 0.9910
Epoch 6/6
359/359 [==============================] - 174s 484ms/step - loss: 0.0036 - acc: 0.9992 - val_loss: 0.0220 - val_acc: 0.9920
CPU times: user 39min 31s, sys: 2min 53s, total: 42min 25s
Wall time: 17min 38s
